In [16]:
import os

def remove_before_and_after(text, start_marker, end_marker):
    # Find the position of the start marker in the text
    start_index = text.find(start_marker)
    
    # If the start marker is found, remove everything before it
    if start_index != -1:
        text = text[start_index + len(start_marker):].strip()
    
    # Find the position of the end marker in the text
    end_index = text.find(end_marker)
    
    # If the end marker is found, remove everything after it
    if end_index != -1:
        text = text[:end_index].strip()
    
    return text

# Function to read text files
def read_text_file(file_path):
    start_marker = " ***"
    end_marker = "*** "
    with open(file_path, 'r', encoding='utf-8') as f:
        file_content = f.read()  # Read the entire file content
        cleaned_content = remove_before_and_after(file_content, start_marker, end_marker)  # Remove before start and after end marker
        return cleaned_content

# Function to iterate over files in the given directory and read text files
def read_books(path):
    # Change the working directory to the specified path
    os.chdir(path)
    
    books_contents = []  # List to store the content of each file
    
    # Iterate through all files in the directory
    for file in os.listdir():
        # Check whether file is in text format or not
        if file.endswith(".txt"):
            # Construct the full file path using os.path.join
            file_path = os.path.join(path, file)
            
            # Call the function to read the text file and append the cleaned content to the list
            books_contents.append(read_text_file(file_path))
    
    return books_contents  # Return the list of book contents


In [18]:
# Example usage 
path = r"C:\Users\ronan\OneDrive - Atlantic TU\Desktop\Documents\GitHub\Emerging-Technologies\tasks\project_gutenberg"
books = read_books(path)

# Now `books` is a list of strings, each representing the cleaned content of a .txt file
for i, book in enumerate(books, start=1):
    print(f"Book {i}:\n{book}\n{'='*40}")

'[Il': 163
'Ill': 169
'llu': 262
'lus': 309
'ust': 1865
'str': 1790
'tra': 1549
'rat': 1288
'ati': 2821
'tio': 3940
'ion': 5057
'on]': 88
'n]
': 87
']

': 170
'


': 891
'

A': 428
'
Al': 110
'Ali': 401
'lic': 773
'ice': 1416
'ce’': 18
'e’s': 683
'’s ': 2836
's A': 84
' Ad': 33
'Adv': 13
'dve': 52
'ven': 1486
'ent': 6035
'ntu': 251
'tur': 1922
'ure': 2009
'res': 2915
'es ': 4413
's i': 2527
' in': 11855
'in ': 8940
'n W': 23
' Wo': 76
'Won': 13
'ond': 905
'nde': 2300
'der': 2076
'erl': 324
'rla': 51
'lan': 1214
'and': 17395
'nd
': 1492
'd

': 4
'

b': 3
'
by': 91
'by ': 2447
'y L': 40
' Le': 181
'Lew': 5
'ewi': 79
'wis': 501
'is ': 9289
's C': 47
' Ca': 567
'Car': 63
'arr': 937
'rro': 406
'rol': 216
'oll': 947
'll
': 474
'l

': 2
'

T': 650
'
TH': 2
'THE': 13
'HE ': 11
'E M': 5
' MI': 1
'MIL': 1
'ILL': 4
'LLE': 4
'LEN': 3
'ENN': 2
'NNI': 1
'NIU': 1
'IUM': 1
'UM ': 2
'M F': 1
' FU': 1
'FUL': 2
'ULC': 1
'LCR': 1
'CRU': 1
'RUM': 1
'M E': 2
' ED': 1
'EDI': 1
'DIT': 1
'ITI': 1
'TIO': 1
'ION